In [1]:
import json
import numpy as np
import time
from PIL import Image
import math
import matplotlib.pyplot as plt
import random
import uuid
import pandas as pd

def f01(x,y,x0,y0,x1,y1,x2,y2):
    q = (x0*y1)-(x1*y0)
    u = y0-y1
    v = x1-x0
    
    return ((u*x) + (v*y) + q)/((u*x2) + (v*y2) + q)

def f12(x,y,x0,y0,x1,y1,x2,y2):
    q = (x1*y2)-(x2*y1)
    u = y1-y2
    v = x2-x1
    
    return ((u*x) + (v*y) + q)/((u*x0) + (v*y0) + q)

def f20(x,y,x0,y0,x1,y1,x2,y2):
    q = (x2*y0)-(x0*y2)
    u = y2-y0
    v = x0-x2
    return ((u*x) + (v*y) + q)/((u*x1) + (v*y1) + q)

In [2]:
# Direction: P2-P1 (128, 100, -1) - (0, 0, 0) = (128, 100, -1)
# Normalize the above
class LightSource:
    def __init__(self, color, intensity, l_type, p=None):
        self.p = p
        self.color = color
        self.intensity = intensity
        self.l_type = l_type
    
# Lighting Model to get ADS 
class Lighting:
    def __init__(self,
                 camera,
                 ambient_light,
                 directional_light
                ):
        self.L = self.normalize(directional_light.p) # TODO: include from-to explicitly incase data is different in future
        self.V = camera.d
        
        # Ambient lighting parameters
        self.aColor = np.array(ambient_light.color)
        self.Ia = np.array(ambient_light.intensity)
        
        # Directional lighting parameters
        self.dColor = np.array(directional_light.color) #Directional Color
        self.Ie = directional_light.intensity
        
    def calculateADS(self, p, n, material):
        n = self.normalize(n) #Normal Vector
        v = self.normalize(self.V - np.array(p))
        
        # Cases that come up in shading...
        if np.dot(n, self.L) < 0 and np.dot(n, v) < 0: n*=-1 # CASE 1: 
        elif np.dot(n, self.L) * np.dot(n, v) < 0: emissive = np.array([0,0,0])
        
        Ka, Kd, Ks = material['Ka'], material['Kd'], material['Ks']
        S = material['n']
        
        a = self.ambient(Ka)
        d = self.diffuse(Kd, n)
        s = self.specular(Ks, n, v, S)
        
        return a+d+s
            
    def ambient(self, Ka):
        return Ka * self.aColor * self.Ia
    
    def diffuse(self, Kd, n):
        dotnl = np.dot(n, self.L)
        dotnl = np.clip(dotnl, 0, 1)
        return Kd * self.dColor * dotnl * self.Ie
    
    def specular(self, Ks, n, v, S):
        r = self.reflectionVector(n)
        
        dotre = np.dot(r, v)
        dotre = np.clip(dotre, 0, 1) # Clamp RE
        dotre = dotre**S # Apply Shininess
        return Ks * self.dColor * dotre * self.Ie
    
    def reflectionVector(self, n):
        # Calculate Normalized Reflection Vector
        r = 2*(np.dot(n, self.L))*n - self.L
        return self.normalize(r)
        
    def normalize(self, pt): 
        l = math.sqrt(pt[0]**2 + pt[1]**2 + pt[2]**2)
        return np.array([pt[0]/l, pt[1]/l, pt[2]/l])

In [3]:
class RenderTransforms:
    def __init__(self):
        # RST Matrix Placeholders
        self.M = np.array([
                            [1, 0, 0, 0],
                            [0, 1, 0, 0],
                            [0, 0, 1, 0],
                            [0, 0, 0, 1]
                        ])
        self.M_inv = np.array([
                            [1, 0, 0, 0],
                            [0, 1, 0, 0],
                            [0, 0, 1, 0],
                            [0, 0, 0, 1]
                        ])
    
    def composeM(self,transforms):
        # Theta assumed in Degrees
        thetax, thetay, thetaz = np.array([transforms[0]['Rx'], transforms[0]['Ry'], transforms[0]['Rz']])*math.pi/180
        tx, ty, tz = transforms[2]['T']
        sx, sy, sz = transforms[1]['S']
        
        trans_mat = self.constructTranslationMat(tx,ty,tz)
        rot_mat = self.constructRotationMat(thetax,thetay,thetaz)
        scale_mat = self.constructScaleMat(sx,sy,sz)
        
        self.M = np.matmul(np.matmul(trans_mat, rot_mat), scale_mat)
        self.M_inv = np.matmul(np.matmul(trans_mat, rot_mat), np.linalg.inv(scale_mat))
        
        return self.M
        
    def constructRotationMat(self, theta_x, theta_y, theta_z):
        XFORM_ROTX = np.array([
                        [1, 0, 0, 0],
                        [0, math.cos(theta_x), -math.sin(theta_x), 0],
                        [0, math.sin(theta_x), math.cos(theta_x), 0 ],
                        [0, 0, 0, 1]
                     ])

        XFORM_ROTY = np.array([
                                [math.cos(theta_y), 0, math.sin(theta_y), 0],
                                [0, 1, 0, 0],
                                [-math.sin(theta_y), 0, math.cos(theta_y), 0],
                                [0, 0, 0, 1],
                             ])

        XFORM_ROTZ = np.array([
                                [math.cos(theta_z), -math.sin(theta_z), 0, 0],
                                [math.sin(theta_z), math.cos(theta_z), 0, 0],
                                [0, 0, 1, 0],
                                [0, 0, 0, 1],
                             ])
        
        rot_mat = np.matmul(XFORM_ROTX, XFORM_ROTY)
        return np.matmul(rot_mat, XFORM_ROTZ)
        
    def constructScaleMat(self, sx, sy, sz):
        return np.array([
                            [sx, 0, 0, 0],
                            [0, sy, 0, 0],
                            [0, 0, sz, 0],
                            [0, 0, 0, 1]
                        ])
    
    def constructTranslationMat(self, tx, ty, tz):
         return np.array([
                             [1, 0, 0, tx],
                             [0, 1, 0, ty],
                             [0, 0, 1, tz],
                             [0, 0, 0, 1]
                        ])
        
    def RST(self, pt):
        pt = np.array(pt+[1])
        return np.matmul(self.M, pt)

# RayTracer
https://bytes.usc.edu/cs580/s22_CG-012-Ren/lectures/Lect_RT/GeomQueries/slides_intersections.html

In [4]:
# Miscellaneous:
#  Determine image plane given camera @ (0,0,0) with a Field-of-View and how to 
#    convert the image plane into a pixelated image. 
#    Let Field-of-View = 120 deg (matching iPhone 13)
#  Create surface plane to show shadows from ray-tracing
#  Calculate sphere model (pending)
 

# Direction: P2-P1 (128, 100, -1) - (0, 0, 0) = (128, 100, -1)
# Normalize the above
class Ray:
    def __init__(self, p0, d, t = -1):
        # p0, origin of a Ray
        self.p0 = p0 
        # Divide d by magnitude of d to normalize
        self.d = d/np.sqrt(np.dot(d,d))
        self.t = t
        # d, unit vector, direction of ray (normalized)
        # self.var = None
    
    def set_t(self, t):
        self.t = t
    
    def get_point(self):
        return np.add(self.p0, np.multiply(self.d, self.t))

class Plane:
    def __init__(self, p0, normal):
        self.p0 = p0 # point on the plane
        self.normal = normal
    
    def on_plane(self, p1):
        # Checks if point is on plane, not really useful but for reference
        return (np.dot(normal, np.subtract(self.p0-p1)) == 0)


# viewport of [1,1] and distance of 1 would make FOV 53 degrees
class RayTracer: 
    def __init__(self, obj_file, scene, lm, tm):        
        self.viewport = scene.viewport
        self.canvas = scene.canvas
        self.camera = scene.camera
        self.triangle = scene.camera
        self.lightModel = lm
        self.transformer = tm
        
        with open(obj_file) as jf:
            data = json.load(jf)
        self.triangles = data['data']
        
    def set_ray(self, d, p=None):
        p0 = self.camera.p0
        if p is not None: p0 = p 
            
        self.ray = Ray(p0, d)
        
    def get_intersect(self, triangle):
        # Calculate vectors
        v0, v1, v2 = [triangle[v]["v"] for v in ["v0", "v1", "v2"]]
        self.v0, self.v1, self.v2 = np.array(v0), np.array(v1), np.array(v2)
        v10 = self.v1 - self.v0
        v20 = self.v2 - self.v0
        
        # Get cross product of vectors 
        n = np.cross(v10, v20)
        self.n = n / np.sqrt(np.dot(n, n))
        d = -np.dot(self.n, self.v0)
        
        self.intersect = -(np.dot(self.n, self.ray.p0) + d)/np.dot(self.n, self.ray.d)
        self.intersect_point = self.ray.d*self.intersect + self.ray.p0
        return self.intersect
    
    def inTriangle(self):
        # The normals for all 3 planes can be encoded in the identity matrix
        plane_normals = np.identity(3)
        # Check for planes perpendicular to the triangle
        planes = np.matmul(plane_normals, self.n)
        # Placeholder
        projected_plane = 0 
        # Select a valid plane to project to
        for i in range(len(planes)): 
            if planes[i] != 0:
                # Just leave loop once you found a valid plane
                projected_plane = i
                break
        
        # Projecting a point onto the x, y, or z plane can be thought of as just 
        # setting, x=0, y=0, or z=0 respectively
        a,b = np.delete(self.intersect_point, projected_plane)
        a0,b0 = np.delete(self.v0, projected_plane)
        a1,b1 = np.delete(self.v1, projected_plane)
        a2,b2 = np.delete(self.v2, projected_plane)
        
        # Calculate Barycentric Coordinates
        self.alpha = f12(a,b,a0,b0,a1,b1,a2,b2)
        self.beta  = f20(a,b,a0,b0,a1,b1,a2,b2)
        self.gamma = f01(a,b,a0,b0,a1,b1,a2,b2)
        
        if (self.alpha >= 0) and (self.beta >= 0) and (self.gamma >= 0): return True
        return False
    
    def castShadowRay(self, light_source, tm, triangles):
        # Attempt to trace shadow ray back to light source 
        # If hits another poylgon, color should be a shadow
        # If shadow ray can be traced back to light source, calculate lighting to fill pixel
        
        d = light_source.p - self.intersect_point
        ray = Ray(ip, d)
        raytracer.set_ray(d, ip)
        
        for triangle in triangles[::20]:
            for v in ["v0", "v1", "v2"]: triangle[v]["v"] = list(tm.RST(triangle[v]["v"]))[:3]
            n = np.array([triangle[v]['n'] for v in ['v0', 'v1', 'v2']])
        
            t = raytracer.get_intersect(triangle)
            if t>0 and raytracer.inTriangle():
                return True
        return False

class Camera:
    def __init__(self, p0, d):
        self.d = d/np.dot(d, d)
        self.p0 = p0

class Viewport:
    def __init__(self, w, h, fov):
        self.a = w/h
        self.x = self.a*math.tan(fov/2)
        self.y = math.tan(fov/2)
        self.z = -1

class Canvas:
    def __init__(self, w, h):
        self.w = w
        self.h = h
        self.z_buffer = np.ones((w, h)) * Z_BUFFER_MAX
        
        

class Scene:
    def __init__(self, camera, viewport, canvas):
        # Camera obj
        self.camera = camera
        # viewport = Viewport obj
        self.viewport = viewport
        # canvas = Canvas obj
        self.canvas = canvas
        # self.lights = lights

    def canvasToViewport(self, cx, cy):
        # Convert a point on the canvas to viewport units
        vw = self.viewport.x
        vh = self.viewport.y
        cw = self.canvas.w
        ch = self.canvas.h

        vx = cx * vw/cw
        vy = cy * vh/ch
        vz = viewport.z

        return [vx, vy, vz]

    def viewportToCanvas(self, vx, vy):
        vw = self.viewport.x
        vh = self.viewport.y
        cw = self.canvas.w
        ch = self.canvas.h
        
        cx = vx*cw/vw
        cy = vy*ch/vh
        
        return cx, cy
    
def biLerp(x, y, x1, y1, img):
    x2, y2 = x1+1, y1+1
    
    q11 = (x1, y1)
    q12= (x1, y2)
    q21 = (x2, y1)
    q22 = (x2, y2)
    
    c11 = np.array(img.getpixel(q11))
    c12 = np.array(img.getpixel(q12))
    c21 = np.array(img.getpixel(q21))
    c22 = np.array(img.getpixel(q22))
    
    xd = x2-x1
    x2d = x2-x
    xd1 = x-x1
    f01 = (x2d/xd)*c11 + (xd1/xd)*c21
    f02 = (x2d/xd)*c12 + (xd1/xd)*c22
    
    yd = y2-y1
    y2d = y2-y
    yd1 = y-y1 
    
    f00 = (y2d/yd)*f01 + (yd1/yd)*f02
    
    return f00

def textureLookup(img, alpha, beta, gamma, t, z):
    uv = (alpha*t[0]) + (beta*t[1]) + (gamma*t[2]) 
    overz = (alpha/z[0]) + (beta/z[1]) + (gamma/z[2])
    uvz = 1/overz
    uv *= uvz

    img_res = img.size
    x = uv[0] * (img_res[0]-1)
    y = uv[1] * (img_res[1]-1)
    
    xt = math.floor(x)
    yt = math.floor(y)
    
    rgb = biLerp(x,y,xt,yt, img)
    
    return rgb


In [8]:
# Render function stolen from Kevin's HW ty 
Z_BUFFER_MAX = np.inf
fov = 120
img_size = 512
skip_triangles = 2 # Render faster

obj_file = "teapot.json"
texture_file = "jupiter.png"

ambient_light_data = {'id': 'L1', 
                      'type': 'ambient', 
                      'color': [1, 1, 1], 
                      'intensity': 0.2}
directional_light_data = {'type': 'directional',
                          'color': [1, 0.5, 1],
                          'intensity': 1.0,
                          'from': [0, 0, 0.1],
                          'to': [0, 0, 0]}
camera_location = [0, 0, 0]
camera_direction = [0, 0, -1]

camera = Camera(camera_location, camera_direction)
viewport = Viewport(img_size, img_size, fov*math.pi/180)
canvas = Canvas(img_size, img_size)

scene = Scene(camera, viewport, canvas)
# texture_map = Image.open(texture_file).convert('RGB')
amb_source = LightSource(ambient_light_data['color'], ambient_light_data['intensity'], "ambient")
dir_source = LightSource(directional_light_data['color'], directional_light_data['intensity'], "directional", directional_light_data["from"])
lm = Lighting(camera, amb_source, dir_source)
tm = RenderTransforms()

raytracer = RayTracer(obj_file, scene, lm, tm)

def render(json_file, save_img=0):
    with open(json_file) as jf:
        data = json.load(jf)
    triangles = data['data']
    material = {'Cs': [0, 1, 0], 'Ka': 0.35, 'Kd': 0.5, 'Ks': 0.9, 'n': 5.0}
    
    img = Image.new('RGB', (canvas.w, canvas.h), 0x0A834E)

    triangle = {'v0': {'v': [1.4, 2.25, 0.0],
            'n': [-0.902861, -0.429934, 0.0],
            't': [0.0, 0.0]},
           'v1': {'v': [1.2915, 2.25, 0.5495],
            'n': [-0.833024, -0.43081, -0.347093],
            't': [0.25, 0.0]},
           'v2': {'v': [1.273482, 2.323828, 0.541834],
            'n': [-0.918898, 0.095044, -0.382874],
            't': [0.25, 0.25]}}
    
    transforms = [{'Rx': 0, 'Ry': 0, 'Rz': 0},
                  {'S': [1, 1, 1]},
                  {'T': [0, 0, -5]}]
    
    triangle = {"v0": {"v": [-1, -1, 0],
                      "n":[0, 0, 1]}, 
                "v1": {"v": [-1, 1, 0],
                      "n":[0, 0, 1]}, 
                "v2": {"v": [1, 0, 0],
                      "n":[0, 0, 1]}
               }
    
    tm.composeM(transforms)
    for v in ["v0", "v1", "v2"]: triangle[v]["v"] = list(tm.RST(triangle[v]["v"]))[:3]
    n = np.array([triangle[v]['n'] for v in ['v0', 'v1', 'v2']])

    
    for i, vx in enumerate(np.linspace(-viewport.x, viewport.x, 512)):
        for j, vy in enumerate(np.linspace(-viewport.y, viewport.y, 512)):
            for triangle in triangles[::skip_triangles]:

                for v in ["v0", "v1", "v2"]: triangle[v]["v"] = list(tm.RST(triangle[v]["v"]))[:3]

                z = [triangle[v]['v'][2] for v in ["v0", "v1", "v2"]]
                t = [np.array(triangle[vz[0]]['t'])/vz[1] for vz in [('v0', z[0]), ('v1', z[1]), ('v2', z[2])]]
                n = np.array([triangle[v]['n'] for v in ['v0', 'v1', 'v2']])

                if min(z) < canvas.z_buffer[i, j]:       
                    d = [vx, vy, viewport.z]
                    raytracer.set_ray(d)

                    t = raytracer.get_intersect(triangle)
                    if t>0 and raytracer.inTriangle() and raytracer.intersect_point[2] < canvas.z_buffer[i, j]:
                        canvas.z_buffer[i, j] = raytracer.intersect_point[2]
                        alpha, beta, gamma = raytracer.alpha, raytracer.beta, raytracer.gamma

    #                     color = textureLookup(texture_map, alpha, beta, gamma, t, z)
    #                         shadow = raytracer.castShadowRay(dir_source, tm, triangles)
    #                         if not shadow:
                        n_interp = (alpha*n[0]) + (beta*n[1]) + (gamma*n[2])
                        ads = lm.calculateADS(t, n_interp, material)
                        rgb = (np.array(ads)* np.array([255, 0, 0])).astype(int)

                        img.putpixel((i, j), tuple(rgb))
    return img

In [9]:
render("teapot.json")

KeyboardInterrupt: 

In [ ]:
class RayTracer: 
    def __init__(self):
        self.none = None
        
    def aFunction(self):
        return 0
    
    def get_intersect(self, ray, v0, v1, v2):
        # Calculate vectors
        self.v0, self.v1, self.v2 = np.array(v0), np.array(v1), np.array(v2)
        v10 = v1 - v0
        v20 = v2 - v0
        
        # Get cross product of vectors 
        n = np.cross(v10, v20)
        self.n = n / np.sqrt(np.dot(n, n))
        
        self.intersect = -(np.dot(self.n, ray.p0) + ray.d)/np.dot(self.n, ray.d)
        return self.intersect
    
    def inTriangle(self):
        # The normals for all 3 planes can be encoded in the identity matrix
        plane_normals = np.identity(3)
        # Check for planes perpendicular to the triangle
        planes = np.matmul(plane_normals, self.n)
        # Placeholder
        projected_plane = 0 
        # Select a valid plane to project to
        for i in range(len(planes)): 
            if planes[i] != 0:
                # Just leave loop once you found a valid plane
                projected_plane = i
                break
        
        # Projecting a point onto the x, y, or z plane can be thought of as just 
        # setting, x=0, y=0, or z=0 respectively
        a,b = np.delete(self.intersect, projected_plane)
        a0,b0 = np.delete(self.v0, projected_plane)
        a1,b1 = np.delete(self.v1, projected_plane)
        a2,b2 = np.delete(self.v2, projected_plane)
        
        # Calculate Barycentric Coordinates
        alpha = f12(a,b,a0,b0,a1,b1,a2,b2)
        beta  = f20(a,b,a0,b0,a1,b1,a2,b2)
        gamma = f01(a,b,a0,b0,a1,b1,a2,b2)
        
        if (self.alpha >= 0) and (self.beta >= 0) and (self.gamma >= 0): return True
        return False

In [ ]:
# Lighting Model to get ADS 
class Lighting:
    def __init__(self,
                 camera,
                 light
                ):
        self.L = self.normalize(light[1]['from']) # TODO: include from-to explicitly incase data is different in future
        self.V = self.normalize(camera['from'])
        
        # Ambient lighting parameters
        self.aColor = np.array(light[0]['color'])
        self.Ia = light[0]['intensity']
        
        # Directional lighting parameters
        self.dColor = np.array(light[1]['color']) #Directional Color
        self.Ie = light[1]['intensity']
        
    def calculateADS(self, n, material):
        n = self.normalize(n)
        # Cases that come up in shading...
        if np.dot(n, self.L) < 0 and np.dot(n, self.V) < 0: n*=-1 # CASE 1: 
        elif np.dot(n, self.L) * np.dot(n, self.V) < 0: emissive = np.array([0,0,0])
        
        Ka, Kd, Ks = material['Ka'], material['Kd'], material['Ks']
        S = material['n']
        
        a = self.ambient(Ka)
        d = self.diffuse(Kd, n)
        s = self.specular(Ks, n, S)
        
        return a+d+s
            
    def ambient(self, Ka):
        return Ka * self.aColor * self.Ia
    
    def diffuse(self, Kd, n):
        dotnl = np.dot(n, self.L)
        dotnl = np.clip(dotnl, 0, 1)
        return Kd * self.dColor * dotnl * self.Ie
    
    def specular(self, Ks, n, S):
        r = self.reflectionVector(n)
        
        dotre = np.dot(r, self.V)
        dotre = np.clip(dotre, 0, 1) # Clamp RE
        dotre = dotre**S # Apply Shininess
        return Ks * self.dColor * dotre * self.Ie
    
    def reflectionVector(self, n):
        # Calculate Normalized Reflection Vector
        r = 2*(np.dot(n, self.L))*n - self.L
        return self.normalize(r)
        
    def normalize(self, pt): 
        l = math.sqrt(pt[0]**2 + pt[1]**2 + pt[2]**2)
        return np.array([pt[0]/l, pt[1]/l, pt[2]/l])

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b1d9b2dc-0aa6-4b59-8bd7-0b3848a2d4a6' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>